### 파킨슨병 데이터
- 환자들의 뇌를 촬영한 사진의 상태를 기록한 자료에 각 환자의 상태 status(1: 파킨슨병 진단, 0: 파킨슨병 아님)로 추가한 테이블
- (data/parkinsons.csv)
1. 파킨슨 병을 예측하는 모델로 로지스틱 회귀모형을 적용하여 생성
2. 파킨슨병을 예측하는데 영향을 미치는 변수를 중요한 순서대로 3개 선정
3. 파킨슨 병을 진단하는 기준를 함수로 생성하여(매개변수명 = threshold, 함수명 = cutoff)을 0.5로 했을 때와 0.8로 했을 때 F1-스코어를 비교
    - 분석 조건
        - 필요 없는 컬럼 name을 삭제
        - 데이터의 정규화는 min-max 스케일러 사용
        - 로지스틱 회귀를 위한 상수항 추가
        - status는 카테고리 타입으로 변환
        - 트레이닝셋과 테스트셋 비율은 9:1
        - 모델은 로지스틱 회귀분석 사용
        - 모델의 최적화 방법론은 "bfgs" 사용

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_pk=pd.read_csv("../datasets/parkinsons.csv")


In [3]:
df_pk.drop('name',axis=1,inplace=True)
df_pk['status']=df_pk['status'].astype('category')

In [4]:
df_pk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   MDVP:Fo(Hz)       195 non-null    float64 
 1   MDVP:Fhi(Hz)      195 non-null    float64 
 2   MDVP:Flo(Hz)      195 non-null    float64 
 3   MDVP:Jitter(%)    195 non-null    float64 
 4   MDVP:Jitter(Abs)  195 non-null    float64 
 5   MDVP:RAP          195 non-null    float64 
 6   MDVP:PPQ          195 non-null    float64 
 7   Jitter:DDP        195 non-null    float64 
 8   MDVP:Shimmer      195 non-null    float64 
 9   MDVP:Shimmer(dB)  195 non-null    float64 
 10  Shimmer:APQ3      195 non-null    float64 
 11  Shimmer:APQ5      195 non-null    float64 
 12  MDVP:APQ          195 non-null    float64 
 13  Shimmer:DDA       195 non-null    float64 
 14  NHR               195 non-null    float64 
 15  HNR               195 non-null    float64 
 16  status            195 non-

In [5]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

x=df_pk.drop('status',axis=1).values
y=df_pk['status'].values
x_trian,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,stratify=y)

In [6]:
Scaler=MinMaxScaler()
Scaler.fit(x_trian)
x_train_sc=Scaler.transform(x_trian)

x_train_sc.max()

1.0000000000000002

In [7]:
from sklearn.linear_model import LogisticRegression

logistic=LogisticRegression(multi_class='multinomial', solver='lbfgs',fit_intercept=True)
logistic.fit(x_train_sc,y_train)

LogisticRegression(multi_class='multinomial')

In [8]:
Scaler.fit(x_test)
x_test_sc=Scaler.transform(x_test)

x_test_sc.max()

1.0000000000000002

In [9]:
sorted_indices = np.argsort(logistic.coef_)
top_3_indices = sorted_indices[0, -3:]
top_3_indices

array([20, 21, 18], dtype=int64)

In [10]:
df_pk.drop('status',axis=1).columns[top_3_indices]

Index(['D2', 'PPE', 'spread1'], dtype='object')

#### PPE ,spread1, D2 가 가장 큰 영향을 끼친다

In [11]:
# 3. 파킨슨 병을 진단하는 기준를 함수로 생성하여(매개변수명 = threshold, 함수명 = cutoff)을 0.5로 했을 때와 0.8로 했을 때 F1-스코어를 비교
from sklearn.metrics import f1_score
from sklearn.preprocessing import Binarizer
pred_proba=logistic.predict_proba(x_test_sc)
pred_proba1=pred_proba[:,1].reshape(-1,1)

def cutoff(threshold):
    binarizer=Binarizer(threshold=threshold).fit(pred_proba1)
    custom_predict=binarizer.transform(pred_proba1)

    F1_score=f1_score(y_test,custom_predict)
    return F1_score
  


In [15]:
cutoff(0.5)

0.9375

In [13]:
cutoff(0.8)

0.7857142857142856